## imports

In [96]:
import matplotlib
matplotlib.use('TKagg')
import matplotlib.pyplot as plt
from matplotlib.pyplot import *
from matplotlib.patches import Polygon
from matplotlib.collections import PatchCollection
import numpy as np
from scipy.spatial import Delaunay
import math
import random

## fonctions de dessins

In [97]:
def draw_diagonals(polygon, diagonals):
    # Draw the polygon
    plt.figure(figsize=(8,8))
    plt.gca().set_aspect('equal')
    x, y = zip(*polygon)
    plt.plot(x + (x[0],), y + (y[0],), '-o')

    # Draw the diagonals
    for i, j in diagonals:
        x, y = zip(polygon[i], polygon[j])
        plt.plot(x, y, 'r')
    
    # Show the plot
    plt.show()


In [98]:
def draw_polygon_and_triangles(points, triangles, zoom_out=1):
    # Créer la figure et l'axe
    fig, ax = plt.subplots()

    # Dessiner le polygone
    poly = Polygon(points, facecolor='white', edgecolor='black')
    ax.add_patch(poly)

    # Dessiner chaque triangle avec une couleur aléatoire
    for triangle in triangles:
        color = (random.random(), random.random(), random.random())
        tri = Polygon(triangle, facecolor=color, edgecolor='black')
        ax.add_patch(tri)

    # Définir les limites de l'axe
    margin = max(1, max(abs(point[0]) for point in points), max(abs(point[1]) for point in points))
    ax.set_xlim([-margin*zoom_out, margin*zoom_out])
    ax.set_ylim([-margin*zoom_out, margin*zoom_out])

    # Afficher la figure
    plt.show()

## Essais Successifs

##### Fonction pour vérifier si une corde est valide La fonction `validecorde_exact` prend en entrée une corde `corde` à tester, une liste de cordes déjà dessinées `drawn_chords` et le nombre de sommets `n` du cycle. Elle renvoie `True` si la corde est valide et `False` sinon.
##### La fonction vérifie si la corde relie deux sommets différents, si elle n'est pas déjà dessinée, si elle ne relie pas des sommets adjacents et si elle ne croise aucune corde déjà dessinée.


In [99]:
def validecorde_exact(corde, drawn_chords,n):
    (i,j) = corde
     # Si i et j sont égaux, alors la corde n'est pas valide car elle relie un sommet à lui-même
    if i==j :
        return False
    # Si i et j sont adjacents, alors la corde n'est pas valide car elle relie des sommets voisins
    # On utilise le modulo n pour considérer le cas où j=i+1=n ou i=j+1=n
    if i == (j+1)%n or j == (i+1)%n : 
        return False
    # Vérifier s'il y a une corde entre i,j qui croise une corde déjà dessinée
    for (k, p) in drawn_chords:
        if (k < i and i < p < j) or (k > i and i > p > j):
            return False
        # Vérifier si la corde i,j est déjà dessinée
        if (i,j) == (k,p) or (j,i) == (k,p) :
            return False
    # On vérifie si la corde (i,j) croise une corde déjà dessinée, 
    # en testant si les sommets k et p d'une corde dessinée (k,p) sont tels que 
    # l'un des deux soit strictement entre i et j dans l'ordre des sommets du cycle, c'est-à-dire k<i<j<p ou p<i<j<k.

    return True
    # Si on arrive jusqu'ici, alors la corde (i,j) ne croise aucune corde déjà dessinée et est donc valide.

#### Essais Successifs
#####
# Stratégie de triangulation de polygones

##### La fonction `generate_best_triangulation` utilise une approche par force brute pour générer toutes les triangulations possibles du polygone d'entrée `C`. Pour ce faire, elle génère d'abord toutes les diagonales possibles du polygone convexe `C` à l'aide de la fonction `generate_diagonals_convex`. Ensuite, pour chaque diagonale possible, elle génère une triangulation du polygone en utilisant la fonction `generate_triangulation_eval`.

##### La fonction `generate_triangulation_eval` utilise un algorithme pour générer une triangulation du polygone à partir de la diagonale de départ donnée. L'algorithme ajoute des diagonales au polygone tant qu'elles ne croisent pas d'autres diagonales déjà tracées et qu'elles ne créent pas de triangles concaves. L'algorithme s'arrête lorsqu'il n'est plus possible d'ajouter de diagonales au polygone.

##### La fonction `generate_best_triangulation` applique ensuite la fonction `generate_triangulation_eval` à chaque diagonale possible du polygone convexe et stocke la triangulation et son évaluation dans deux listes distinctes. Enfin, elle renvoie la triangulation qui a la plus petite évaluation.

##### En somme, la stratégie utilisée par la fonction `generate_best_triangulation` consiste à générer toutes les triangulations possibles du polygone d'entrée et à les évaluer pour trouver celle qui minimise la longueur totale des diagonales. Bien que cette approche soit exhaustive, elle est garantie de trouver la meilleure triangulation possible. Cependant, cette méthode peut s'avérer coûteuse en temps de calcul pour les polygones de grande taille.

In [100]:
def generate_diagonals_convex(points):
    """
    Cette fonction génère toutes les diagonales convexes d'un polygone donné en utilisant une approche par force brute. 
    Elle prend en entrée une liste de tuples représentant les points du polygone et renvoie une liste de tuples contenant 
    les indices des points reliés par les diagonales.
    """
    diagonales = []
    n = len(points)
    for i in range(n):
        for j in range(i+2, n):
            diagonales.append((points[i], points[j]))
    diagonales_index = [ (points.index(p[0]),points.index(p[1])) for p in diagonales]
    diagonales_index.remove((0,n-1))
    return diagonales_index

# exemple d'utilisation:
points = [(0, 0), (1, 1), (2, 0), (1, -1)]
C = [(-1, 1), (-1, 3), (1, 4), (3, 3), (2, 1), (0, 0)]
diagonales = generate_diagonals_convex(C)

#print(diagonals, len(diagonals))
#O(n^2)

In [101]:
def line_length(p1, p2):
    """
    Cette fonction calcule la distance euclidienne entre deux points 2D donnés en entrée p1 et p2 et renvoie un nombre flottant.
    """
    x1, y1 = p1
    x2, y2 = p2
    return math.sqrt((x2 - x1) ** 2 + (y2 - y1) ** 2)

In [102]:
def generate_triangulation_eval(C, diagonales, start):
    """
    Cette fonction génère une triangulation du polygone donné en utilisant les diagonales données en entrée et en partant de 
    la diagonale de départ 'start'. Elle renvoie une liste de tuples contenant les indices des points reliés par les diagonales ainsi 
    que la longueur totale de la triangulation.
    """
    n = len(C)
    diagonales_dessinées = []
    eval = line_length(C[start[0]], C[start[1]])
    diagonales_dessinées.append(start)
    i = 0
    while len(diagonales_dessinées) < n-3 or i<(n-3)*(n-2)/2:
        if validecorde_exact(diagonales[i], diagonales_dessinées, n):
            diagonales_dessinées.append(diagonales[i])
            eval += line_length(C[diagonales[i][0]], C[diagonales[i][1]])
        i += 1
    return diagonales_dessinées, eval 

#C = [(-1, 1), (-1, 3), (1, 4), (3, 3), (2, 1), (0, 0)]
#diagonals = generate_diagonals_convex(C)
#print(generate_triangulation_eval(C,diagonals,(0,2)))

In [103]:
def generate_best_triangulation(C):
    """
    Cette fonction génère toutes les triangulations possibles du polygone donné et renvoie la triangulation avec la 
    longueur totale minimale. Elle utilise les fonctions generate_diagonals_convex et generate_triangulation_eval.
    """
    diagonales_possibles = generate_diagonals_convex(C)
    triangulations = []
    evals = []  
    for i in diagonales_possibles:  
        returns = generate_triangulation_eval(C, diagonales_possibles, i)
        triangulations +=  [returns[0]]
        evals += [returns[1]]
    return triangulations[evals.index(min(evals))]

In [104]:
C = [
(180.49111067193468, 494.6643076028935)
,(153.57298273821428, 491.8990116382506)
,(124.90587499022688, 482.56823390821756)
,(94.8304013738639, 465.4617906982039)
,(79.65802095744523, 451.0048215906153)
,(58.657721967764864, 430.7563139713908)
,(22.125366180142493, 377.135118374759)
,(15.018875463509772, 340.6585145248268)
,(7.423964246997095, 237.6564116158071)
,(13.25404024697252, 177.78175687888677)
,(24.105947793609307, 125.83176826111064)
,(42.27962105864147, 78.94556462003533)
,(55.46466046939314, 46.68742678288962)
,(65.52681116466663, 33.29498410064791)
,(126.6137140781215, 23.464199508610008)
,(282.78977610022685, 9.617583587439471)
,(356.9265394776163, 7.778598135969372)
,(363.79596068265454, 9.146574531481711)
,(363.8598081309624, 9.168552444235868)
,(410.2550243605333, 28.412895732593867)
,(446.6594395588621, 47.50671199284218)
,(460.78665921087867, 64.044003818305)
,(475.03239783670716, 92.81868127186299)
,(465.8151124650581, 164.52408991617017)
,(454.9461041438435, 219.66632833107658)
,(327.8984491216154, 442.4023448567188)
,(285.1837042223283, 490.3952039723944)
,(281.86092750835576, 490.7914741816579)
,(276.3170853336246, 491.370066028904)]
draw_diagonals(C, generate_best_triangulation(C))

#### test

## Dynamique

##### L'algorithme de triangulation de polygones suivant est basé sur la méthode de programmation dynamique. Il consiste à trouver des triangles formés par les sommets du polygone de manière à le découper en triangles non intersectants. Pour cela, l'algorithme cherche des "oreilles" du polygone, c'est-à-dire des triangles formés par trois sommets qui ne contiennent pas d'autres sommets du polygone à l'intérieur. Une fois une oreille trouvée, elle est enlevée du polygone et le processus est répété sur le polygone restant jusqu'à ce qu'il ne reste plus de sommets. L'algorithme retourne alors la liste des triangles formés. 


In [105]:
def triangulate_polygon(polygon):
    triangles = []
    n = len(polygon)
    
    # Si le nombre de sommets du polygone est inférieur à 3, il ne peut pas être triangulé
    if n < 3:
        return triangles
    
    # Parcourt tous les sommets du polygone
    for i in range(n):
        prev = (i - 1) % n
        curr = i
        next = (i + 1) % n
        
        # Vérifie si le triangle formé par les sommets i, i+1 et i+2 est une "oreille" du polygone
        if is_ear(polygon, prev, curr, next):
            # Ajoute le triangle à la liste des triangles et enlève le sommet i+1 du polygone
            triangles.append((polygon[prev], polygon[curr], polygon[next]))
            polygon.pop(curr)
            # Répète le processus de triangulation sur le polygone restant
            return triangles + triangulate_polygon(polygon)
    
    # Si aucun triangle n'a été trouvé pour trianguler le polygone, la triangulation est terminée
    return triangles

def is_ear(polygon, i, j, k):
    # Vérifie si le triangle formé par les sommets i, j et k ne contient pas d'autres sommets du polygone
    for m in range(len(polygon)):
        if m != i and m != j and m != k and is_inside_triangle(polygon[m], polygon[i], polygon[j], polygon[k]):
            return False
    return True

def is_inside_triangle(p, a, b, c):
    # Vérifie si le point p est à l'intérieur du triangle formé par les sommets a, b et c
    return is_same_side(p, a, b, c) and is_same_side(p, b, a, c) and is_same_side(p, c, a, b)

def is_same_side(p1, p2, a, b):
    # Vérifie si les points p1 et p2 sont du même côté de la ligne formée par les sommets a et b
    cp1 = cross_product(b[0] - a[0], b[1] - a[1], p1[0] - a[0], p1[1] - a[1])
    cp2 = cross_product(b[0] - a[0], b[1] - a[1], p2[0] - a[0], p2[1] - a[1])
    return cp1 * cp2 >= 0

def cross_product(x1, y1, x2, y2):
    # Calcule le produit vectoriel de deux vecteurs
    return x1 * y2 - x2 * y1

#### test

In [106]:
C = [
(180.49111067193468, 494.6643076028935)
,(153.57298273821428, 491.8990116382506)
,(124.90587499022688, 482.56823390821756)
,(94.8304013738639, 465.4617906982039)
,(79.65802095744523, 451.0048215906153)
,(58.657721967764864, 430.7563139713908)
,(22.125366180142493, 377.135118374759)
,(15.018875463509772, 340.6585145248268)
,(7.423964246997095, 237.6564116158071)
,(13.25404024697252, 177.78175687888677)
,(24.105947793609307, 125.83176826111064)
,(42.27962105864147, 78.94556462003533)
,(55.46466046939314, 46.68742678288962)
,(65.52681116466663, 33.29498410064791)
,(126.6137140781215, 23.464199508610008)
,(282.78977610022685, 9.617583587439471)
,(356.9265394776163, 7.778598135969372)
,(363.79596068265454, 9.146574531481711)
,(363.8598081309624, 9.168552444235868)
,(410.2550243605333, 28.412895732593867)
,(446.6594395588621, 47.50671199284218)
,(460.78665921087867, 64.044003818305)
,(475.03239783670716, 92.81868127186299)
,(465.8151124650581, 164.52408991617017)
,(454.9461041438435, 219.66632833107658)
,(327.8984491216154, 442.4023448567188)
,(285.1837042223283, 490.3952039723944)
,(281.86092750835576, 490.7914741816579)
,(276.3170853336246, 491.370066028904)]
triangles = triangulate_polygon(C)
draw_polygon_and_triangles(C, triangles)

## Greedy

##### Ce code implémente une méthode de triangulation d'un polygone convexe en utilisant une méthode de programmation gloutonne. L'idée est de choisir la corde extérieure la plus courte à chaque étape de la triangulation.

##### La fonction `triangle_greedy` prend en entrée une liste de points qui représentent les sommets du polygone convexe. Elle retourne une liste de triangles, chaque triangle étant représenté par un tuple de 3 points. 

In [107]:
def triangle_greedy(points):
    # Vérifier que les points forment un polygone convexe
    n = len(points)
    if n < 3:
        # Si la liste de points contient moins de 3 points, elle ne peut pas former un polygone convexe,
        # donc on retourne une liste vide
        return []

    # Initialiser la liste des triangles
    triangles = []

    # Tant qu'il reste des sommets à trianguler
    while n > 2:
        # Rechercher la corde extérieure la plus courte
        min_distance = float('inf')
        min_index = -1
        for i in range(n):
            p1 = points[i]
            p2 = points[(i+1)%n]
            for j in range(1, n-1):
                if j == i or j == (i+1)%n:
                    continue
                p3 = points[(i+j)%n]
                p4 = points[(i+j+1)%n]
                distance = line_length(p1, p3) + line_length(p2, p4)
                if distance < min_distance:
                    min_distance = distance
                    min_index = (i+j)%n

        # Ajouter le triangle formé par la corde extérieure
        triangles.append((points[min_index], points[(min_index+1)%n], points[(min_index-1)%n]))

        # Retirer le point sélectionné
        points.pop(min_index)
        n -= 1

    # Retourner la liste des triangles
    return triangles

#### test

In [108]:
C = [
(180.49111067193468, 494.6643076028935)
,(153.57298273821428, 491.8990116382506)
,(124.90587499022688, 482.56823390821756)
,(94.8304013738639, 465.4617906982039)
,(79.65802095744523, 451.0048215906153)
,(58.657721967764864, 430.7563139713908)
,(22.125366180142493, 377.135118374759)
,(15.018875463509772, 340.6585145248268)
,(7.423964246997095, 237.6564116158071)
,(13.25404024697252, 177.78175687888677)
,(24.105947793609307, 125.83176826111064)
,(42.27962105864147, 78.94556462003533)
,(55.46466046939314, 46.68742678288962)
,(65.52681116466663, 33.29498410064791)
,(126.6137140781215, 23.464199508610008)
,(282.78977610022685, 9.617583587439471)
,(356.9265394776163, 7.778598135969372)
,(363.79596068265454, 9.146574531481711)
,(363.8598081309624, 9.168552444235868)
,(410.2550243605333, 28.412895732593867)
,(446.6594395588621, 47.50671199284218)
,(460.78665921087867, 64.044003818305)
,(475.03239783670716, 92.81868127186299)
,(465.8151124650581, 164.52408991617017)
,(454.9461041438435, 219.66632833107658)
,(327.8984491216154, 442.4023448567188)
,(285.1837042223283, 490.3952039723944)
,(281.86092750835576, 490.7914741816579)
,(276.3170853336246, 491.370066028904)]
triangles = triangle_greedy(C)
draw_polygon_and_triangles(C, triangles)